In [37]:
!pip install ultralytics

In [38]:
## importing required libraries
import os
import shutil
import random

!pip install tqdm --upgrade
from tqdm.notebook import tqdm

In [39]:
## connecting to the google drive

from google.colab import drive
drive.mount('/content/drive')



Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [40]:

train_path_img = "/content/drive/MyDrive/yolov8/train/images"
train_path_label = "/content/drive/MyDrive/yolov8/train/labels"
val_path_img = "/content/drive/MyDrive/yolov8/valid/images"
val_path_label = "/content/drive/MyDrive/yolov8/valid/labels"
test_path = "/content/drive/MyDrive/yolov8/test_images"

In [41]:
'''
Split the dataset into train and test and creates the train.txt and test.tx with
the respective path of the images in each folder
'''

import os
import random

def train_test_split(path,neg_path=None, split = 0.2):
    print("------ PROCESS STARTED -------")


    files = list(set([name[:-4] for name in os.listdir(path)])) ## removing duplicate names i.e. counting only number of images


    print (f"--- This folder has a total number of {len(files)} images---")
    random.seed(42)
    random.shuffle(files)

    test_size = int(len(files) * split)
    train_size = len(files) - test_size

    ## creating required directories

    os.makedirs(train_path_img, exist_ok = True)
    os.makedirs(train_path_label, exist_ok = True)
    os.makedirs(val_path_img, exist_ok = True)
    os.makedirs(val_path_label, exist_ok = True)


    ### ----------- copying images to train folder
    for filex in tqdm(files[:train_size]):
      if filex == 'classes':
          continue
      shutil.copy2(path + filex + '.jpg',f"{train_path_img}/" + filex + '.jpg' )
      shutil.copy2(path + filex + '.txt', f"{train_path_label}/" + filex + '.txt')




    print(f"------ Training data created with 80% split {len(files[:train_size])} images -------")

    if neg_path:
        neg_images = list(set([name[:-4] for name in os.listdir(neg_path)])) ## removing duplicate names i.e. counting only number of images
        for filex in tqdm(neg_images):
            shutil.copy2(neg_path+filex+ ".jpg", f"{train_path_img}/" + filex + '.jpg')
            shutil.copy2(path + filex + '.txt', f"{train_path_label}/" + filex + '.txt')

        print(f"------ Total  {len(neg_images)} negative images added to the training data -------")

        print(f"------ TOTAL Training data created with {len(files[:train_size]) + len(neg_images)} images -------")



    ### copytin images to validation folder
    for filex in tqdm(files[train_size:]):
      if filex == 'classes':
          continue
      # print("running")
      shutil.copy2(path + filex + '.jpg', f"{val_path_img}/" + filex + '.jpg' )
      shutil.copy2(path + filex + '.txt', f"{val_path_label}/" + filex + '.txt')

    print(f"------ Testing data created with a total of {len(files[train_size:])} images ----------")

    print("------ TASK COMPLETED -------")

## spliting the data into train-test and creating train.txt and test.txt files
# train_test_split('/content/drive/MyDrive/custom_notebooks/yolo_data/')

### for label_tag
train_test_split('/content/drive/MyDrive/yolov8/data/') ### without negative images
# train_test_split('./data/','./negative_images/')  ### if you want to feed negative images


------ PROCESS STARTED -------
--- This folder has a total number of 240 images---


  0%|          | 0/192 [00:00<?, ?it/s]

------ Training data created with 80% split 192 images -------


  0%|          | 0/48 [00:00<?, ?it/s]

------ Testing data created with a total of 48 images ----------
------ TASK COMPLETED -------


In [42]:

import ultralytics
ultralytics.checks()

Ultralytics YOLOv8.0.148 🚀 Python-3.10.12 torch-2.0.1+cu118 CUDA:0 (Tesla T4, 15102MiB)
Setup complete ✅ (2 CPUs, 12.7 GB RAM, 24.5/78.2 GB disk)


In [43]:
!yolo task=detect mode=train model=yolov8s.pt data=/content/drive/MyDrive/yolov8/data.yaml epochs=10 imgsz=640 batch=8 project=/content/drive/MyDrive/yolov8/training_results name=retail

100% 21.5M/21.5M [00:00<00:00, 92.1MB/s]
Ultralytics YOLOv8.0.148 🚀 Python-3.10.12 torch-2.0.1+cu118 CUDA:0 (Tesla T4, 15102MiB)
engine/trainer: task=detect, mode=train, model=yolov8s.pt, data=/content/drive/MyDrive/yolov8/data.yaml, epochs=10, patience=50, batch=8, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=/content/drive/MyDrive/yolov8/training_results, name=retail, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, show=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, vid_stride=1, line_width=None, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_

In [44]:
!yolo task=detect mode=predict model=/content/drive/MyDrive/yolov8/training_results/retail/weights/best.pt conf=0.55 source=/content/drive/MyDrive/yolov8/test_images

Ultralytics YOLOv8.0.148 🚀 Python-3.10.12 torch-2.0.1+cu118 CUDA:0 (Tesla T4, 15102MiB)
Model summary (fused): 168 layers, 11164284 parameters, 0 gradients

image 1/10 /content/drive/MyDrive/yolov8/test_images/0aaecdd8-3eb7-4b23-90c7-d7efe71f9ed4_jpeg_jpg.rf.33e4bbdcd4ac31ee37688412b16a1e4d.jpg: 640x640 (no detections), 16.6ms
image 2/10 /content/drive/MyDrive/yolov8/test_images/11c07914-607a-401e-9282-b378d5d14eeb_jpeg_jpg.rf.9301f7f27a0c6d6f7d1fd12e64e26c92.jpg: 640x640 (no detections), 16.7ms
image 3/10 /content/drive/MyDrive/yolov8/test_images/1f765667-8d5d-4100-8a75-c1d5012dfd35_jpeg_jpg.rf.d57f074f1d6ddc9a32223d10a1e8fa13.jpg: 640x640 (no detections), 16.6ms
image 4/10 /content/drive/MyDrive/yolov8/test_images/27cc00e8-b9fb-43ad-8667-4eb0be286a2c_jpeg_jpg.rf.d973659e211134de2c2febe119bec4fc.jpg: 640x640 (no detections), 16.6ms
image 5/10 /content/drive/MyDrive/yolov8/test_images/3f3e800e-7475-415f-a0cb-871cb86a67e5_jpeg_jpg.rf.b49c43911212790c4f0ae5f4b332afcc.jpg: 640x640 (no det

In [45]:
!cp -r /content/runs/detect/predict /content/drive/MyDrive/yolov8/output